
[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


작업 폴더: /Users/serinoh/SAPA


원본 데이터: 23,679명 × 719변수
채점 키: 696문항 × 131척도


QC 1 - 응답 부족 제외:
  제외 전: 23,679명
  제외 후: 23,678명
  제외 인원: 1명


In [29]:
# QC 2: Straight-lining 제외 (모든 응답이 동일한 경우)
def is_straight_lining(row, item_cols):
    """응답이 모두 같은 값인지 확인"""
    responses = row[item_cols].dropna()
    if len(responses) < 10:
        return False
    return responses.nunique() == 1

straight_lining = df[valid_n_responses].apply(
    lambda row: is_straight_lining(row, item_cols), axis=1
)

valid_qc = valid_n_responses.copy()
valid_qc.loc[valid_qc[valid_qc].index[straight_lining]] = False

print(f'\nQC 2 - Straight-lining 제외:')
print(f'  제외 전: {valid_n_responses.sum():,}명')
print(f'  제외 후: {valid_qc.sum():,}명')
print(f'  제외 인원: {straight_lining.sum():,}명')

# 최종 유효 데이터
df_valid = df[valid_qc].copy()
print(f'\n최종 유효 N: {len(df_valid):,}명')


QC 2 - Straight-lining 제외:
  제외 전: 23,678명
  제외 후: 23,647명
  제외 인원: 31명

최종 유효 N: 23,647명


Big Five 점수 계산 중...
완료!



=== Big Five 통계량 ===
NEO_O           N=23,399  Mean= 4.31  SD= 0.83
NEO_C           N=23,417  Mean= 4.23  SD= 0.86
NEO_E           N=23,357  Mean= 3.87  SD= 0.92
NEO_A           N=23,437  Mean= 4.23  SD= 0.82
NEO_N           N=23,371  Mean= 3.35  SD= 0.98


=== Ideology 통계량 ===
Ideology      N=11,825  Mean= 0.00  SD= 0.98


In [34]:
# Honesty-Humility 구성 요소 계산
scores['NEO_Morality'] = calculate_scale_score(df_valid, keys, 'NEOa2')
scores['NEO_Modesty'] = calculate_scale_score(df_valid, keys, 'NEOa4')
scores['HEXACO_H'] = calculate_scale_score(df_valid, keys, 'HEXACO_H')

# z-score 계산 (세 구성 요소가 모두 유효한 경우에만 계산)
valid_mask = (scores['NEO_Morality'].notna() & 
              scores['NEO_Modesty'].notna() & 
              scores['HEXACO_H'].notna())

if valid_mask.sum() > 0:
    # 유효한 데이터만 추출하여 z-score 계산
    neo_a2_valid = scores.loc[valid_mask, 'NEO_Morality']
    neo_a4_valid = scores.loc[valid_mask, 'NEO_Modesty']
    hexaco_h_valid = scores.loc[valid_mask, 'HEXACO_H']
    
    z_neo_a2 = pd.Series(stats.zscore(neo_a2_valid.values), index=neo_a2_valid.index)
    z_neo_a4 = pd.Series(stats.zscore(neo_a4_valid.values), index=neo_a4_valid.index)
    z_hexaco_h = pd.Series(stats.zscore(hexaco_h_valid.values), index=hexaco_h_valid.index)
    
    # Honesty-Humility = mean(z(NEOa2), z(NEOa4), z(HEXACO_H))
    scores['Honesty_Humility'] = (z_neo_a2 + z_neo_a4 + z_hexaco_h) / 3
else:
    scores['Honesty_Humility'] = np.nan

print('=== Honesty-Humility 통계량 ===')
n = scores['Honesty_Humility'].notna().sum()
mean = scores['Honesty_Humility'].mean()
std = scores['Honesty_Humility'].std()
print(f'Honesty_Humility  N={n:6,}  Mean={mean:5.2f}  SD={std:5.2f}')

=== Honesty-Humility 통계량 ===
Honesty_Humility  N=12,828  Mean= 0.00  SD= 0.74



=== 전체 척도 통계량 요약 ===
척도                          N     Mean       SD
---------------------------------------------
NEO_O                  23,399     4.31     0.83
NEO_C                  23,417     4.23     0.86
NEO_E                  23,357     3.87     0.92
NEO_A                  23,437     4.23     0.82
NEO_N                  23,371     3.35     0.98
Ideology               11,825     0.00     0.98
Honesty_Humility       12,828     0.00     0.74

총 계산된 응답자 수: 23,647명


✅ 저장 완료: data/processed/sapa_scores.csv
저장된 척도: 7개
저장된 응답자 수: 23,647명
